---

Some useful packages and libraries:



In [273]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time

---

## Problem 1: Game Theory - Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>



### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [274]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

        
class TicTacToe:
    
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.nexp = nexp

    def result(self, move, state):
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):       
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        turn_limit = self.nrow*self.ncol 
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    self.display()
                    return self.state.utility                

### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [275]:
def random_player(game):
    move = random.choice(game.state.moves)
    return move


We know from experience and/or because I'm telling you right now that if two `random_player`s play many games of Tic-Tac-Toe against one another, whoever goes first will win about 58% of the time.  Verify that this is the case by playing at least 1,000 games between two random players. Report the proportion of the games that the first player has won.

**"Unit tests":** If you are wondering how close is close enough to 58%, I simulated 100 tournaments of 1,000 games each. The min-max range of win percentage by the first player was 54-63%.

In [276]:
"Simulate 1,000 games"
num_games = 1000
x_wins = 0
o_wins = 0
draws = 0  

for _ in range(num_games):
    game = TicTacToe()
    result = game.play_game(random_player, random_player)
    if result == 1:
        print("      X wins")
        print("\n")
        x_wins += 1
    elif result == -1:
        print("      O wins")
        print("\n")
        o_wins += 1
    else:
        print("      Draw")
        print("\n")
        draws += 1

x_win_percentage = (x_wins / num_games) * 100
o_win_percentage = (o_wins / num_games) * 100
draw_percentage = (draws / num_games) * 100

print(f"Out of {num_games} games:")
print(f"'X' won {x_win_percentage}% of the time.")
print(f"'O' won {o_win_percentage}% of the time.")
print(f"The games were a draw {draw_percentage}% of the time.")



. X X 
. X O 
O X O 
      X wins


X O X 
X X O 
O X O 
      Draw


O O X 
X X O 
X . . 
      X wins


O O O 
X X O 
. X X 
      O wins


X X O 
X O O 
O . X 
      O wins


O X . 
X O . 
. X O 
      O wins


X X . 
O O O 
X X O 
      O wins


. X X 
O O O 
. . X 
      O wins


X . X 
. X O 
X O O 
      X wins


X O O 
X X X 
O X O 
      X wins


O X X 
X O O 
O X X 
      Draw


X . . 
X O O 
X O X 
      X wins


X O . 
O X O 
X . X 
      X wins


X O X 
X O O 
O X X 
      Draw


X O O 
X . . 
X . . 
      X wins


X O . 
X O O 
X X . 
      X wins


X X O 
X . O 
O X O 
      O wins


X . O 
X . O 
X . . 
      X wins


O X O 
. X X 
. X O 
      X wins


X O X 
O X O 
X X O 
      X wins


O O X 
. O X 
X X O 
      O wins


O O X 
. O X 
X X O 
      O wins


O X O 
O X X 
X X O 
      X wins


X X O 
X O O 
. X O 
      O wins


O X X 
X X O 
O O X 
      Draw


X O O 
X X O 
X . . 
      X wins


X O X 
X O O 
X X O 
      X wins


O . X 
O O X 
X X O 
      O wins




### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **1b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*


<span style="color:red">The 4x4 expansion results in the increased number of possible moves and configurations of Xs and Os on the board. With more possibilities for move sequences, the likelihood of either player securing a win before the board fills up lessens, which leads to more draws. The expanded state space also introduces a more balanced playing field for both players. Since there are more opportunities to block the opponent and more spaces to consider for strategic play, the first-player advantage that is prominent in 3x3 Tic-Tac-Toe is less pronounced in the 4x4 version. This balance contributes to an equalization of win and loss rates between the two players.</span>


   


In [277]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

        
class TicTacToe4x4:
    
    def __init__(self, nrow=4, ncol=4, nwin=4, nexp=0): # 4x4 board
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        self.nexp = nexp

    def result(self, move, state):
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):      
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    self.display()
                    return self.state.utility     
                
"Simulate 1,000 games"
num_games = 1000
x_wins = 0
o_wins = 0
draws = 0  

for _ in range(num_games):
    game = TicTacToe4x4()
    result = game.play_game(random_player, random_player)
    if result == 1:
        print("      X wins")
        print("\n")
        x_wins += 1
    elif result == -1:
        print("      O wins")
        print("\n")
        o_wins += 1
    else:
        print("      Draw")
        print("\n")
        draws += 1

# Calculate percentages
x_win_percentage = (x_wins / num_games) * 100
o_win_percentage = (o_wins / num_games) * 100
draw_percentage = (draws / num_games) * 100

print(f"Out of {num_games} games:")
print(f"'X' won {x_win_percentage}% of the time.")
print(f"'O' won {o_win_percentage}% of the time.")
print(f"The games were a draw {draw_percentage}% of the time.")

X X O X 
X O O X 
X X O O 
O O O X 
      O wins


X O X O 
O O X O 
X O . X 
X O . X 
      O wins


O X O O 
. O O O 
X X X X 
O X X X 
      X wins


X O O O 
X X O O 
X X X O 
O X X O 
      O wins


X O X O 
X O X X 
X O O X 
O O O X 
      O wins


X O O X 
O O X X 
X X O O 
X . . . 
      X wins


O X . O 
. X O O 
O X X O 
X X . X 
      X wins


O X O X 
X O X O 
O O X X 
X X O O 
      Draw


O O O X 
X O X X 
X O O X 
X . O X 
      X wins


O O O O 
X . . O 
O X X X 
X . X . 
      O wins


O X X O 
O O X O 
X X X X 
O X . O 
      X wins


O O X O 
X O O O 
O X X X 
X O X X 
      Draw


O X X O 
X O O X 
O X O X 
O O X X 
      Draw


O O X X 
O X O X 
X X X O 
X O O O 
      Draw


. . O X 
X . O X 
. . O O 
X . O X 
      O wins


X O O X 
X X X O 
X X O O 
O X O O 
      Draw


X X . O 
O X X X 
O O O O 
X X . O 
      O wins


O X X X 
X X O X 
O O O O 
X O X O 
      O wins


X O O O 
X O X X 
X X O O 
O X X O 
      Draw


O O O O 
X . X . 
O . X X 
. X X O 
      O

### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [278]:
def alphabeta_player(game):
    """Return the minimax move using alpha-beta pruning."""
    def max_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, state.to_move), None
        v, move = float('-inf'), None
        for m in state.moves:
            v2, _ = min_value(game.result(m, state), alpha, beta)
            if v2 > v:
                v, move = v2, m
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.game_over(state):
            return game.utility(state, state.to_move), None
        v, move = float('inf'), None
        for m in state.moves:
            v2, _ = max_value(game.result(m, state), alpha, beta)
            if v2 < v:
                v, move = v2, m
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    _, move = max_value(game.state, float('-inf'), float('inf'))
    return move

Run the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **10 games for each test**, and track the number of wins, draws and losses. Report these results for each case.

1. An alpha-beta player who plays first versus a random player who plays second.
2. Two alpha-beta players.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. You are welcome to run more than 10 tests if you'd like. 

In [279]:
import random

def run_test(player1, player2, num_games=10):
    results = {'Player 1 Wins': 0, 'Player 2 Wins': 0, 'Draws': 0}
    for _ in range(num_games):
        game = TicTacToe()
        winner = game.play_game(player1, player2)
        if winner == 1:
            results['Player 1 Wins'] += 1
        elif winner == -1:
            results['Player 2 Wins'] += 1
        else:
            results['Draws'] += 1
    return results



#Test 1: Alpha-beta player (X) vs. Random player (O)
results_test_1 = run_test(alphabeta_player, random_player)

# Test 2: Two Alpha-beta players
results_test_2 = run_test(alphabeta_player, alphabeta_player)

print("Test 1 Results:", results_test_1)
print("Test 2 Results:", results_test_2)


X X X 
. . O 
. . O 
X X X 
. . O 
. O . 
X X X 
. O . 
. O . 
X X X 
. O . 
. . O 
X X X 
O . . 
O . . 
X O X 
X O O 
O X X 
X X X 
. . O 
. O . 
X O X 
X O O 
O X X 
X X X 
. O . 
. . O 
X X X 
. . . 
O O . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
X O X 
O X O 
X . . 
Test 1 Results: {'Player 1 Wins': 8, 'Player 2 Wins': 0, 'Draws': 2}
Test 2 Results: {'Player 1 Wins': 10, 'Player 2 Wins': 0, 'Draws': 0}


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Then compute the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [280]:
class TicTacToe:

    def minimax_search(game, state):
        global minimax_counter
        minimax_counter = 0
        
        def max_value(state):
            global minimax_counter
            minimax_counter += 1
            if game.game_over(state):
                return game.utility(state, state.to_move)
            v = float('-inf')
            for m in state.moves:
                v = max(v, min_value(game.result(m, state)))
            return v

        def min_value(state):
            global minimax_counter
            minimax_counter += 1
            if game.game_over(state):
                return game.utility(state, state.to_move)
            v = float('inf')
            for m in state.moves:
                v = min(v, max_value(game.result(m, state)))
            return v
        
        max_value(state)
        return minimax_counter

    def alphabeta_search(game, state):
        global alphabeta_counter
        alphabeta_counter = 0
        
        def max_value(state, alpha, beta):
            global alphabeta_counter
            alphabeta_counter += 1
            if game.game_over(state):
                return game.utility(state, state.to_move)
            v = float('-inf')
            for m in state.moves:
                v = max(v, min_value(game.result(m, state), alpha, beta))
                if v >= beta:
                    return v
                alpha = max(alpha, v)
            return v

        def min_value(state, alpha, beta):
            global alphabeta_counter
            alphabeta_counter += 1
            if game.game_over(state):
                return game.utility(state, state.to_move)
            v = float('inf')
            for m in state.moves:
                v = min(v, max_value(game.result(m, state), alpha, beta))
                if v <= alpha:
                    return v
                beta = min(beta, v)
            return v
        
        max_value(state, float('-inf'), float('inf'))
        return alphabeta_counter

    # Initialize and run the searches
    game = TicTacToe()
    initial_state = game.state  # Assuming this is the starting state of the game

    minimax_nodes_expanded = minimax_search(game, initial_state)
    alphabeta_nodes_expanded = alphabeta_search(game, initial_state)

    percent_lost = (alphabeta_nodes_expanded / minimax_nodes_expanded) * 100
    percent_savings = 100 - percent_lost

    print(f"Minimax nodes expanded: {minimax_nodes_expanded}")
    print(f"Alpha-beta nodes expanded: {alphabeta_nodes_expanded}")
    print(f"Percent savings from pruning: {percent_savings:.2f}%")



Minimax nodes expanded: 549946
Alpha-beta nodes expanded: 2322
Percent savings from pruning: 99.58%


---


## Problem 2:  Bayesian network to model heart disease

The following Bayesian network is based loosely on a study that examined heart disease risk factors in 167 elderly individuals in South Carolina.  Note that this figure uses Y and N to represent Yes and No, whereas in class we used the equivalent T and F to represent True and False Boolean values.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw05_bayesnet_heartdisease.png" style="width: 650px;"/>

### (2a) 

Create a `BayesNet` object to model this.  Below is the code for the (conditional) probability `P` function and `BayesNode` class as well, that we used in our coding notebook to represent the variable nodes and calculate probabilities. You can code this however you want, subject to the following constraints:
1. the nodes are represented using the `BayesNode` class and can work with the `P` function for probabilities,
1. your `BayesNet` structure keeps track of which nodes are in the Bayes net, as well as
1. which nodes are the parents/children of which other nodes.

Some *suggested* skeleton codes for a class structure are given. The suggestions for methods to implement are in view of the fact that we will need to calculate some probabilities, which is going to require us to `find_node`s and `find_values` that nodes can take on.

In [281]:
## For the sake of brevity...
T, F = True, False

## From class:
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row] if value else 1-var.cpt[row]

## Also from class:
class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

    
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by adding each of the nodes,
        which should be a list BayesNode class objects ordered
        from parents to children (`top` to `bottom`, from causes
        to effects)'''
        self.nodes = [] if nodes is None else nodes
        self.variables = [node.variable for node in nodes]  

        for node in self.nodes:
            for parent in node.parents:
                self.find_node(parent).children.append(node)


        
        # your code goes here...

                
    def add(self, node):
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        '''Add a new BayesNode to the BayesNet. The parents should all
        already be in the net, and the variable itself should not be'''
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.find_node(parent).children.append(node)

       


        
        # your code goes here...
        

            
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`'''
        for node in self.nodes:
            if node.variable==var:
                return node
        return None
    
        
        # your code goes here...


        
    def find_values(self, var):
        '''Return the set of possible values for variable `var`'''
        node = self.find_node(var)
        return node.values
    
        
        # your code goes here...
        

    
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

In [282]:
# Define the nodes


smoking = BayesNode('Sm', '', [True, False], 0.2)
exercise = BayesNode('ME', '', [True, False], 0.5)
highbp = BayesNode('HBP', ['Sm', 'ME'], [True, False], {(True, True): 0.6, (True, False): 0.72, (False, True): 0.33, (False, False): 0.51})
family_hist = BayesNode('FH', '', [True, False], 0.15)
atherosclerosis = BayesNode('Ath', '', [True, False], 0.53)
heart_disease = BayesNode('HD', ['HBP', 'Ath', 'FH'], [True, False],
                          {(True, True, True): 0.92, (True, True, False): 0.91, (True, False, True): 0.81,
                           (True, False, False): 0.77, (False, True, True): 0.75, (False, True, False): 0.69,
                           (False, False, True): 0.38, (False, False, False): 0.23})
angina = BayesNode('Ang', ['HD'], [True, False], {True: 0.85, False: 0.4})
rapid_heartbeat = BayesNode('Rapid', ['HD'], [True, False], {True: 0.99, False: 0.3})

# Create a Bayes net with those nodes and connections
nodes = [smoking, exercise, highbp, family_hist, atherosclerosis, heart_disease, angina, rapid_heartbeat]
bn = BayesNet(nodes)





### (2b)

Craft a function `get_prob(X, e, bn)` to return the **normalized** probability distribution of variable `X` in Bayes net `bn`, given the evidence `e`.  That is, return $P(X \mid e)$. The arguments are:
* `X` is some representation of the variable you are querying the probability distribution of. Either a string (the variable name from the `BayesNode` or a `BayesNode` object itself are good options.
* `e` is some representation of the evidence your probability is conditioned on. When given an empty argument (or `None`) for `e`, `get_prob` should return the marginal distribution $P(X)$.
* `bn` is your `BayesNet` object.

You may do this using the `enumeration` algorithm from class (pseudocode is in the book), or by brute force (i.e., use a few `for` loops). Either way, you should be using your `BayesNet` object to keep track of all the nodes and relationships between nodes so your `get_prob` function knows these things.

In [283]:
# Your code here.
def get_prob(X, e, bn):    
    def enumerate_all(vars, e):
        if not vars:
            return 1.0
        Y, rest = vars[0], vars[1:]
        Ynode = bn.find_node(Y)
        if Y in e:
            return P(Ynode, e[Y], e) * enumerate_all(rest, e)
        else:
            return sum(P(Ynode, y, e) * enumerate_all(rest, extend(e, Y, y))
                       for y in Ynode.values)

    def extend(s, var, val):
        s2 = s.copy()
        s2[var] = val
        return s2

    Xnode = bn.find_node(X) if isinstance(X, str) else X
    Q = {x: enumerate_all(bn.variables, extend(e, X, x)) for x in Xnode.values}
    

    total = sum(Q.values())
    return {k: v / total for k, v in Q.items()}

Use your `get_prob` function to calculate the following probabilities. Print them to the screen and compare to the original Bayes net figure given to make sure the output passes these "unit tests".

1. The marginal probability of `Family History` is $P(FH=T)=0.15$
2. The probability of *not* experiencing `Angina Pectoris`, given `Heart Disease` is observed, is $P(Ang=F \mid HD=T)=1-0.85=0.15$
3. The probability of `High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise`, is $P(HBP=T \mid Sm=T, ME=F)=0.72$

In [284]:
# Your code here.
#Marginal probability of `Family History`
prob_FH = get_prob('FH', {}, bn)
print(f"The marginal probability of Family History (FH=T): {prob_FH[True]}")

#Not experiencing `Angina Pectoris`, given `Heart Disease` is observed
prob_Ang_given_HD = get_prob('Ang', {'HD': True}, bn)
print(f"The probability of not experiencing Angina Pectoris (Ang=F | HD=T): {prob_Ang_given_HD[False]}")

#`High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise`
prob_HBP_given_Sm_and_not_ME = get_prob('HBP', {'Sm': True, 'ME': False}, bn)
print(f"The probability of High Blood Pressure (HBP=T | Sm=T, ME=F): {prob_HBP_given_Sm_and_not_ME[True]}")

The marginal probability of Family History (FH=T): 0.15
The probability of not experiencing Angina Pectoris (Ang=F | HD=T): 0.15000000000000002
The probability of High Blood Pressure (HBP=T | Sm=T, ME=F): 0.7199999999999999


### (2c)

Calculate the probability of observing someone with `High Blood Pressure`, $P(HBP=T)$, *by hand*, showing all work in Markdown/LateX below.

$$
\begin{align*}
P(HBP=T) = & 0.6 \cdot 0.2 \cdot 0.5 \\
& + 0.72 \cdot 0.2 \cdot 0.5 \\
& + 0.33 \cdot 0.8 \cdot 0.5 \\
& + 0.51 \cdot 0.8 \cdot 0.5 \\
\end{align*}
$$



$$
\begin{align*}
P(HBP=T) = & 0.6 \cdot 0.1 + 0.72 \cdot 0.1 + 0.33 \cdot 0.4 + 0.51 \cdot 0.4 \\
= & 0.06 + 0.072 + 0.132 + 0.204 \\
= & 0.468 \\
\end{align*}
$$

**Verify** your calculation using your `get_prob` function.

In [285]:
prob_HBP = get_prob('HBP', {}, bn)
print(f"The marginal probability of High Blood Pressure (HBP=T): {prob_HBP[True]}")

The marginal probability of High Blood Pressure (HBP=T): 0.4680000000000001


### (2d)

Now calculate the following probabilities using your `get_prob` function.

[i] The probability of an arbitrary individual having `Heart Disease`, $P(HD=T)$

In [286]:
prob_HD = get_prob('HD', {}, bn)
print(f"The probability of Heart Disease (HD=T): {prob_HD[True]}")

The probability of Heart Disease (HD=T): 0.65700256


[ii] The probability that an individual does *not* have `Heart Disease`, given that `Rapid Heartbeat` was observed, $P(HD=F \mid Rapid=T)$

In [287]:
prob_HD_given_Rapid = get_prob('HD', {'Rapid': True}, bn)
print(f"The probability of not having Heart Disease (HD=F | Rapid=T): {prob_HD_given_Rapid[False]}")

The probability of not having Heart Disease (HD=F | Rapid=T): 0.13659218499670053


[iii] The probability of an individual having `High Blood Pressure` if they have `Heart Disease` and a `Family History`, $P(HBP=T \mid HD=T, FH=T)$

In [288]:
prob_HBP_given_HD_and_FH = get_prob('HBP', {'HD': True, 'FH': True}, bn)
print(f"The probability of High Blood Pressure (HBP=T | HD=T, FH=T): {prob_HBP_given_HD_and_FH[True]}")

The probability of High Blood Pressure (HBP=T | HD=T, FH=T): 0.5700562923792059


[iv] The probability that an individual is a `Smoker/Alcohol User` if they have `Heart Disease`, $P(Sm=T \mid HD=T)$

In [289]:
prob_Sm_given_HD = get_prob('Sm', {'HD': True}, bn)
print(f"The probability of being a Smoker/Alcohol User (Sm=T | HD=T): {prob_Sm_given_HD[True]}")

The probability of being a Smoker/Alcohol User (Sm=T | HD=T): 0.22096327904719273


[v] How would you expect the probability in [iv] to change if you also know the individual has `High Blood Pressure`?  Verify your hypothesis by calculating the relevant probability.

**I would expect it to increase because HBP is positlvey correlated wiht both HD and Sm.**

In [290]:
prob_Sm_given_HD_and_HBP = get_prob('Sm', {'HD': True, 'HBP': True}, bn)
print(f"The probability of being a Smoker/Alcohol User (Sm=T | HD=T, HBP=T): {prob_Sm_given_HD_and_HBP[True]}")


The probability of being a Smoker/Alcohol User (Sm=T | HD=T, HBP=T): 0.282051282051282


[vi] How would you expect the probability in [v] to change if you also know that the individual does *not* get `Moderate Exercise` (in addition to having `Heart Disease` and `High Blood Pressure`)?  Explain your answer using concepts from class.  Verify your answer by calculating the relevant probability.

I would expect it to decrease because, the added evidence (ME=F) dilutes the impact of the original evidence (HD=T and HBP=T) on the probability of Sm=T.

In [291]:
prob_Sm_given_HD_HBP_ME = get_prob('Sm', {'HD': True, 'HBP': True, 'ME': False}, bn)
print(f"The probability of being a Smoker/Alcohol User (Sm=T | HD=T, HBP=T, ME=F): {prob_Sm_given_HD_HBP_ME[True]}")

The probability of being a Smoker/Alcohol User (Sm=T | HD=T, HBP=T, ME=F): 0.26086956521739124


---


## Problem 3:  Bayesian network to model decision-making

Let's consider using a Bayesian network to model our decision about whether or not to ride our bike to work today.  This decision depends heavily on the weather, so let's focus on that.

In class, we focused on Boolean variables.  For example, we might base our biking decision on whether or not it is raining.  But in reality, it probably matters *how hard* it is raining.  So suppose we break the variable `Rain` up into three discrete bins: `none`, `light` and `heavy`.

The temperature also factors into our decision.  There is definitely a sweet spot, where temperatures are neither too warm nor too cold, so it is very likely we would enjoy riding our bike.  So we can model the variable `Temperature` also using three discrete bins: `cold`, `moderate` and `warm`.

So a Bayesian network to model our decision for whether or not to bike to work could be as follows, where the first letter of each discrete bin is used to denote that variable value (i.e., `R=h` stands for heavy rain conditions).

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/bayesnet_biking2.png" style="width: 650px;"/>

### (3a)

Modify the `P` probability function to be able to handle these ternary parent nodes.

In [292]:
def P(var, value, evidence={}):
    if len(var.parents) == 0:
        return var.cpt[value]
    else:
        parent_values = tuple(evidence[parent] for parent in var.parents)
        return var.cpt[parent_values][value]

Set up `BayesNode` objects for each of `Rain`, `Temp` and `Bike`, and create a `BayesNet` object to model the Bayesian network for this decision.  Again, you can use whatever structure you wish for your `BayesNet`, but please use the `BayesNode` class.  You may need to make minor modifications to the `BayesNode` class (e.g., changing/adding attributes), although none are strictly necessary.

In [293]:
class BayesNode2:
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
        
        self.variable = name
        self.parents = parents
        self.values = values
        self.cpt = cpt
        self.children = []
        
    def __repr__(self):
        return f"BayesNode({self.variable})"

class BayesNet:
    def __init__(self, nodes):
        self.nodes = nodes
        self.variables = set(node.variable for node in nodes)
        for node in nodes:
            for parent in node.parents:
                self.find_node(parent).children.append(node)

    def add(self, node):
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        self.nodes.append(node)
        self.variables.add(node.variable)
        for parent in node.parents:
            self.find_node(parent).children.append(node)

    def find_node(self, var):
        for node in self.nodes:
            if node.variable == var:
                return node
        return None

    def find_values(self, var):
        node = self.find_node(var)
        return node.values if node else None

    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)


rain = BayesNode2('R', '', ['n', 'l', 'h'], {'n': 0.8, 'l': 0.15, 'h': 0.05})
temperature = BayesNode2('T', '', ['c', 'm', 'w'], {'c': 0.3, 'm': 0.6, 'w': 0.1})
bike_cpt = {
    ('n', 'c'): {'y': 0.7, 'n': 0.3},
    ('n', 'm'): {'y': 0.99, 'n': 0.01},
    ('n', 'w'): {'y': 0.9, 'n': 0.1},
    ('l', 'c'): {'y': 0.4, 'n': 0.6},
    ('l', 'm'): {'y': 0.6, 'n': 0.4},
    ('l', 'w'): {'y': 0.5, 'n': 0.5},
    ('h', 'c'): {'y': 0.2, 'n': 0.8},
    ('h', 'm'): {'y': 0.4, 'n': 0.6},
    ('h', 'w'): {'y': 0.3, 'n': 0.7},
}
bike = BayesNode2('B', ['R', 'T'], ['y', 'n'], bike_cpt)
bn2 = BayesNet([rain, temperature, bike])



    





**Verify** that your modified probability function `P` is working by checking the following "unit tests". Print the output to screen and compare to what you expect from the figure above.

1. The marginal probability of no rain is $P(Rain=n)=0.8$
1. The marginal probability of light rain is $P(Rain=l)=0.15$
1. The marginal probability of heavy rain is $P(Rain=h)=0.05$
1. The probability of biking given that it is raining heavily and the temperature is cold, is $P(Bike=T \mid Rain=h, Temp=c)=0.2$

In [294]:
#Marginal probability of no rain: P(Rain=n)
prob_rain_n = P(rain, 'n')
print("1. The marginal probability of no rain is:", prob_rain_n)

#Marginal probability of light rain: P(Rain=l)
prob_rain_l = P(rain, 'l')
print("2. The marginal probability of light rain is:", prob_rain_l)

#Marginal probability of heavy rain: P(Rain=h)
prob_rain_h = P(rain, 'h')
print("3. The marginal probability of heavy rain is:", prob_rain_h)

#Probability of biking given heavy rain and cold temperature: P(Bike=T | Rain=h, Temp=c)
prob_bike_given_h_c = P(bike, 'y', {'R': 'h', 'T': 'c'})
print("4. The probability of biking given heavy rain and cold temperature is:", prob_bike_given_h_c)


1. The marginal probability of no rain is: 0.8
2. The marginal probability of light rain is: 0.15
3. The marginal probability of heavy rain is: 0.05
4. The probability of biking given heavy rain and cold temperature is: 0.2


### (3b)

Make any necessary modifications to your `get_prob` function from Problem 2, so that you can use it to calculate marginal probabilities and conditional probabilities for this problem. It is possible that your function does not require any modifications.

In [295]:
#modified get_prob

def get_prob2(X, e, bn):
    def enumerate_all(vars, e):
        if not vars:
            return 1.0
        Y, *rest = vars
        Ynode = bn.find_node(Y)
        if Y in e:
            return P(Ynode, e[Y], e) * enumerate_all(rest, e)
        else:
            return sum(P(Ynode, y, e) * enumerate_all(rest, extend(e, Y, y))
                       for y in Ynode.values)

    def extend(s, var, val):
        s2 = s.copy()
        s2[var] = val
        return s2

    Xnode = bn.find_node(X) if isinstance(X, str) else X
    Q = {x: enumerate_all(bn.variables, extend(e, X, x)) for x in Xnode.values}
    
    total = sum(Q.values())
    return {k: v / total for k, v in Q.items()}


Use `get_prob` to calculate $P(Bike)$, the probability distribution for whether or not you will ride your bike on any given day.

In [296]:
prob_bike = get_prob2('B', {}, bn2)
print("\nMarginal probability of riding the bike:")
print("Bike=Y:", prob_bike['y'])



Marginal probability of riding the bike:
Bike=Y: 0.8112


Use `get_prob` to calculate the probability that you will ride your bike, given that it is lightly raining.

In [297]:
prob_bike_given_rain_l = get_prob2('B', {'R': 'l'}, bn2)
print("\nProbability of riding the bike given light rain:")
print("Bike=Y:", prob_bike_given_rain_l['y'])


Probability of riding the bike given light rain:
Bike=Y: 0.53


### (3c)

We are trapped indoors because some jerk gave us a ton of Intro to Artificial Intelligence homework to do.  Suppose we look out the window and see people biking. They sure do look like they're having fun! *Given* this information, we can actually make inferences regarding the temperature outside!  What is the probability distribution for temperature, given that we observe people biking?

First, compute this using your `get_prob` function.

In [298]:
evidence = {'B': 'y'}  

prob_temperature_given_biking = get_prob2('T', evidence, bn2)
print("\nProbability distribution of temperature given biking:")
for temp, prob in prob_temperature_given_biking.items():
    print(f"Temp={temp}:", prob)



Probability distribution of temperature given biking:
Temp=c: 0.2329881656804734
Temp=m: 0.6671597633136096
Temp=w: 0.09985207100591718


### (3d)

Confirm your answer to **3c** by hand, showing *all* relevant work below in a LateX/Markdown cell.

$$
P(B) = P(B=y) = \sum_{T} P(B=y | T) \cdot P(T)
$$
$$
P(T | B) = \frac{P(B=y | T) \cdot P(T)}{P(B)}
$$

1. \( P(B) \):
$$
P(B=y) = P(B=y | T=c) \cdot P(T=c) + P(B=y | T=m) \cdot P(T=m) + P(B=y | T=w) \cdot P(T=w)
$$
$$
= (0.7 \cdot 0.3) + (0.99 \cdot 0.6) + (0.9 \cdot 0.1)
$$
$$
= 0.21 + 0.594 + 0.09
$$
$$
= 0.894
$$

2. \( P(T | B) \) for each temperature value:
$$
P(T=c | B) = \frac{P(B=y | T=c) \cdot P(T=c)}{P(B)}
$$
$$
= \frac{0.7 \cdot 0.3}{0.894}
$$
$$
\approx 0.2329881656804734
$$

$$
P(T=m | B) = \frac{P(B=y | T=m) \cdot P(T=m)}{P(B)}
$$
$$
= \frac{0.99 \cdot 0.6}{0.894}
$$
$$
\approx 0.6671597633136096
$$

$$
P(T=w | B) = \frac{P(B=y | T=w) \cdot P(T=w)}{P(B)}
$$
$$
= \frac{0.9 \cdot 0.1}{0.894}
$$
$$
\approx 0.09985207100591718
$$

The probability distribution of temperature given that we observe people biking is approximately:
$$
- P(T=c | B) \approx 0.2329881656804734
- P(T=m | B) \approx 0.6671597633136096
- P(T=w | B) \approx 0.09985207100591718
$$



